In [1]:
import numpy as np
import tensorflow as tf
import glob
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [6]:
!pip install unrar
!unrar x /content/drive/MyDrive/dataset/Final_v1.rar

Streaming output truncated to the last 5000 lines.
Extracting  Final_v1/Train/Smoke/006198.jpg                               86%  OK 
Extracting  Final_v1/Train/Smoke/006199.jpg                               86%  OK 
Extracting  Final_v1/Train/Smoke/006200.jpg                               86%  OK 
Extracting  Final_v1/Train/Smoke/006201.jpg                               86%  OK 
Extracting  Final_v1/Train/Smoke/006202.jpg                               86%  OK 
Extracting  Final_v1/Train/Smoke/006203.jpg                               86%  OK 
Extracting  Final_v1/Train/Smoke/006204.jpg                               86%  OK 
Extracting  Final_v1/Train/Smoke/006205.jpg                               86%  OK 
Extracting  Final_v1/Train/Smoke/006206.jpg                               86%  OK 
Extracting  Final_v1/Train/Smoke/006207.jpg                               86%  OK 
Extracting  Final_v1/Train/Sm

In [7]:

train_path='/content/Final_v1/Train'
val_path='/content/Final_v1/Test'

IMAGE_SIZE = (224, 224, 3)
resnet = ResNet152(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
for layer in resnet.layers:
    layer.trainable = True

folders = glob.glob('/content/Final_v1/Test/*')
len(folders)
x = Flatten()(resnet.output)
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=prediction)
#model.summary()​

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [8]:
train_datagen = ImageDataGenerator( horizontal_flip = True,
                                    vertical_flip = True)

val_datagen = ImageDataGenerator(horizontal_flip = True,
                                  vertical_flip = True)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
validation_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 59884 images belonging to 3 classes.
Found 14070 images belonging to 3 classes.


In [9]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping
callbacks = [ModelCheckpoint('.mdl_wts.hdf5', monitor='val_accuracy',mode='max',verbose=1, save_best_only=True),
             ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, verbose=1, mode='max', min_lr=0.00000000001)]
r = model.fit(
  training_set,
  validation_data=validation_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(validation_set),
  callbacks=callbacks
)